# **Preparação do Ambiente e Dados**

In [1]:
import pandas as pd
import numpy as np

# Criando dados fictícios com erros propositais
data = {
    'ID_Pedido': [101, 102, 103, 104, 105, 101], # ID 101 está duplicado
    'Data': ['2023-10-01', '2023-10-01', '01/10/2023', '2023-10-02', '2023-10-02', '2023-10-01'],
    'Produto': ['Widget A', 'Widget B', 'Widget A', 'Widget C', 'Widget B', 'Widget A'],
    'Quantidade': [2, 1, 5, 10, np.nan, 2], # Temos um valor nulo (NaN)
    'Preco_Unitario': [50.0, 100.0, 50.0, 25.0, 100.0, 50.0],
    'Total_Informado': [100.0, 100.0, 250.0, 240.0, 0.0, 100.0] # O ID 104 tem erro de cálculo (10*25 != 240)
}

df_fake = pd.DataFrame(data)
df_fake.to_csv('vendas_bruto.csv', index=False)
print("Arquivo 'vendas_bruto.csv' criado com sucesso!")

Arquivo 'vendas_bruto.csv' criado com sucesso!


## **O Fluxo de Operações (Pipeline)**

In [2]:
import pandas as pd

def processar_operacoes(arquivo_entrada):
    print(f"--- Iniciando Processamento: {arquivo_entrada} ---")

    # 1. Leitura dos dados
    df = pd.read_csv(arquivo_entrada)

    # 2. Limpeza Inicial
    # Remover duplicatas (comum em erros de sistema de vendas)
    duplicatas = df.duplicated().sum()
    df = df.drop_duplicates()

    # Preencher valores nulos em Quantidade com 0 e converter para int
    df['Quantidade'] = df['Quantidade'].fillna(0).astype(int)

    # Padronizar datas (importante para relatórios temporais)
    df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

    # 3. Conciliação e Auditoria (O pulo do gato para Operações)
    # Verificamos se o 'Total_Informado' bate com (Quantidade * Preco_Unitario)
    df['Total_Calculado'] = df['Quantidade'] * df['Preco_Unitario']
    df['Diferenca_Conciliacao'] = df['Total_Calculado'] - df['Total_Informado']

    erros_conciliacao = df[df['Diferenca_Conciliacao'] != 0]

    # 4. Geração de Alertas
    if not erros_conciliacao.empty:
        print(f"⚠️ ALERTA: Encontrados {len(erros_conciliacao)} erros de conciliação financeira!")
        print(erros_conciliacao[['ID_Pedido', 'Total_Informado', 'Total_Calculado']])

    # 5. Relatório Executivo
    relatorio = {
        'Total de Vendas': df['Total_Calculado'].sum(),
        'Qtd Itens Vendidos': df['Quantidade'].sum(),
        'Ticket Médio': df['Total_Calculado'].mean(),
        'Duplicatas Removidas': duplicatas
    }

    return df, relatorio

# Executando a função
df_limpo, resumo = processar_operacoes('vendas_bruto.csv')

print("\n--- Resumo Executivo ---")
for chave, valor in resumo.items():
    print(f"{chave}: {valor:.2f}" if isinstance(valor, float) else f"{chave}: {valor}")

--- Iniciando Processamento: vendas_bruto.csv ---
⚠️ ALERTA: Encontrados 1 erros de conciliação financeira!
   ID_Pedido  Total_Informado  Total_Calculado
3        104            240.0            250.0

--- Resumo Executivo ---
Total de Vendas: 700.00
Qtd Itens Vendidos: 18
Ticket Médio: 140.00
Duplicatas Removidas: 1
